In [105]:
using DataFrames, CSV, NamedArrays
raw = CSV.read("crops1.csv");
# turn Crops into an array
crops_array = convert(Array,raw);
# the `names' of the DataFrame (header) are the information of each crop;
info_crops = names(raw[2:end]);
# create a list of crops from the crops array
crops = crops_array[1:end,1];
# create a NamedArray that specifies the information of each crop
crops=convert(Array{Symbol},crops )
using NamedArrays
crops_matrix = crops_array[1:end,2:end]; # rows are crops, columns are information of crops
crops_information_array = NamedArray(crops_matrix, (crops, info_crops), ("crops","info_crops"));


raw1 = CSV.read("LiveStock1.csv");
# turn LiveStock into an array
lstocks_array = convert(Array,raw1);
# the `names' of the DataFrame (header) are the information of each livestock;
info_lstocks = names(raw1[2:end]);
# create a list of stocks from the livestocks array
lstocks = lstocks_array[1:end,1];
# create a NamedArray that specifies the information of each livestock
lstocks=convert(Array{Symbol},lstocks )
lstocks_matrix = lstocks_array[1:end,2:end] ;# rows are livestocks, columns are information of livestocks
lstocks_information_array = NamedArray(lstocks_matrix, (lstocks, info_lstocks), ("lstocks","info_lstocks"));

#crops to feed our livestocks
feedcrops= [:Wheat, :Corn, :Soybean, :Hay];

#the properties of our farm
tt_field=900;
tt_housing=3;
tt_money=10000000
numcrop=5
numlstock=5
lb_crops = 
lb_lstock = 

using JuMP,Gurobi
m = Model(solver=GurobiSolver(OutputFlag=0)) # create model named m

#variables are the area(acre) to grow each crop and the number(hd) for each livestock
@variable(m, crop[crops] >= 0)
@variable(m, y[crops], Bin)
@variable(m, lstock[lstocks]>=0)
@variable(m, z[lstocks], Bin)

println(crops_information_array)
println()
println(crops)
println(lstocks)
println(info_lstocks)

#constraints of pasture and field
@constraint(m, sum(crop[i] for i in crops) +sum(lstock[j]*lstocks_information_array[j,:Pasture] for j in lstocks)<= tt_field )

#constraints of housing area
@constraint(m, tthousing, sum(lstock[j]*lstocks_information_array[j,:Enclosed_Housing] for j in lstocks) <= tt_housing )

#constraint for money available
@constraint(m, sum(crop[i]*crops_information_array[i,:Cost] for i in crops) + sum(lstock[j]*lstocks_information_array[j,:Cost] for j in lstocks) <= tt_money)

# if crop[i]>0, then y[i]=1
@constraint(m,div1[i in crops], crop[i]<=y[i]*tt_field)

# if lstock[i]>0, then z[i]=1
@constraint(m, div2[j in lstocks], lstock[j]*lstocks_information_array[j,:Cost]<=z[j]*tt_money)

#diversity constraint for crop
@constraint(m, sum(y[i] for i in crops)==numcrop)

#diversity constraint for livestock
@constraint(m,sum(z[j]for j in lstocks)>=numlstock)

#constraints for feeding the livestocks using our own crops
#@constraint(m, constr[k in feedcrops],sum(lstock[j]*lstocks_information_array[j,k] for j in lstocks) <= crops_information_array[k,:Return] )

#expression for the total cost of livestock
@expression(m, tot_cost_lstock, sum(lstock[i] * lstocks_information_array[i,:Cost] for i in lstocks))
#expression fpr the total cost of crop
@expression(m,   tot_cost_crop,sum(crop[i] * crops_information_array[i,:Cost] for i in crops))
#expression for the total return from livestocks
@expression(m, gross_return_lstock, sum(lstock[i]* lstocks_information_array[i,:Return ] for i in lstocks))
#expression fpr the total return from crops
@expression(m, gross_return_crop, sum(crop[i]* crops_information_array[i,:Return] for i in crops))

#objective: to maximize the return
@objective(m, Max, gross_return_crop + gross_return_lstock - tot_cost_lstock - tot_cost_crop)

solve(m)
println(getobjectivevalue(m))
println(getvalue(crop))
println(getvalue(lstock))
println(getvalue(y))
println(getvalue(z))


12×2 Named Array{Any,2}
crops ╲ info_crops │   :Cost  :Return
───────────────────┼─────────────────
:Corn              │  257.65   565.12
:Soybean           │  158.14  386.908
:Wheat             │  159.57  217.147
:Oat               │  119.38  143.144
:Barley            │  135.24   323.07
:Flaxseed          │  130.77   142.41
:Rye               │  225.65  161.025
:Peas              │  116.83   229.69
:Beans             │  239.63   401.96
:Canola            │  187.14   327.18
:Sunflower         │  233.22   580.68
:Hay               │  147.28   322.08

Symbol[:Corn, :Soybean, :Wheat, :Oat, :Barley, :Flaxseed, :Rye, :Peas, :Beans, :Canola, :Sunflower, :Hay]
Symbol[:Yearling_Steers, :Steers_Calves, :Dairy_Goat, :Pig, :Sheep, :Broiler_organic, :Broiler_regular]
Symbol[:Enclosed_Housing, :Exercise_Yard, :Pasture, :Corn, :Soybean, :Wheat, :Hay, :Other_Costs, :Return, :Net_Return, :Cost]
Academic license - for non-commercial use only
312727.93999999994
crop: 1 dimensions:
[     Corn] = 0.0
[  